In [1]:
!git clone 'https://github.com/aakashvardhan/s7-in-depth-practice.git'

Cloning into 's7-in-depth-practice'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 27 (delta 7), reused 25 (delta 5), pack-reused 0
Receiving objects: 100% (27/27), 85.70 KiB | 2.86 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [2]:
import sys
sys.path.append('/content/s7-in-depth-practice')
sys.path.append('/content/s7-in-depth-practice/models')

In [3]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from model_utils import model_summary, test_model_sanity
from models.model import model1

from utils import train, test

In [4]:
testmodel = model1()
test_model_sanity(testmodel)

100%|██████████| 9912422/9912422 [00:00<00:00, 106029382.31it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 96599436.86it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 31950859.75it/s]


...

Epoch 2


Loss: 2.3025 Batch_id=9: 100%|██████████| 10/10 [00:00<00:00, 36.37it/s]


Epoch 3


Loss: 2.3026 Batch_id=9: 100%|██████████| 10/10 [00:00<00:00, 23.46it/s]

Sanity check passed: Model is capable of overfitting to a small subset of the data.


In [5]:
# Train Phase transformations
train_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values.
                                       # Note the difference between (0.1307) and (0.1307,)
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])

In [6]:
train_data = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test_data = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

In [7]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train_data, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test_data, **dataloader_args)

CUDA Available? True


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [8]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model1 = model1().to(device)
model_summary(model1, (1, 28, 28))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]              90
              ReLU-2           [-1, 10, 26, 26]               0
            Conv2d-3           [-1, 10, 24, 24]             900
              ReLU-4           [-1, 10, 24, 24]               0
            Conv2d-5           [-1, 20, 22, 22]           1,800
              ReLU-6           [-1, 20, 22, 22]               0
         MaxPool2d-7           [-1, 20, 11, 11]               0
            Conv2d-8           [-1, 10, 11, 11]             200
              ReLU-9           [-1, 10, 11, 11]               0
           Conv2d-10             [-1, 10, 9, 9]             900
             ReLU-11             [-1, 10, 9, 9]               0
           Conv2d-12             [-1, 20, 7, 7]           1,800
             ReLU-13             [-1, 20, 7, 7]               0
           Conv2d-14             [

In [9]:
optimizer = optim.SGD(model1.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model1, device, train_loader, optimizer, epoch)
    test(model1, device, test_loader)

EPOCH: 0


Loss=1.184149146080017 Batch_id=468 Accuracy=21.07: 100%|██████████| 469/469 [00:16<00:00, 28.89it/s]



Test set: Average loss: 1.0713, Accuracy: 6457/10000 (64.57%)

EPOCH: 1


Loss=0.4730045795440674 Batch_id=468 Accuracy=87.64: 100%|██████████| 469/469 [00:15<00:00, 30.08it/s]



Test set: Average loss: 0.2639, Accuracy: 9199/10000 (91.99%)

EPOCH: 2


...


Test set: Average loss: 0.0521, Accuracy: 9836/10000 (98.36%)

EPOCH: 13


Loss=0.046542346477508545 Batch_id=468 Accuracy=98.55: 100%|██████████| 469/469 [00:16<00:00, 29.28it/s]



Test set: Average loss: 0.0505, Accuracy: 9851/10000 (98.51%)

EPOCH: 14


Loss=0.03244422748684883 Batch_id=468 Accuracy=98.63: 100%|██████████| 469/469 [00:15<00:00, 29.47it/s]



Test set: Average loss: 0.0523, Accuracy: 9834/10000 (98.34%)

